Scales


In [2]:
import pandas as pd

#lets create a dataframe of letter grades in descending order. we can also set an
#index value and here we'll just make it some human judgement of how good a student
#was

df = pd.DataFrame(['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D'],
                    index = ['excellent', 'excellent', 'excellent', 'good', 'good', 'good', 
                       'ok', 'ok', 'ok', 'poor', 'poor'],
                       columns = ['Grades'])
df                       

,Grades
excellent,A+
excellent,A
excellent,A-
good,B+
good,B
good,B-
ok,C+
ok,C
ok,C-
poor,D+


In [3]:
#note: we'll see that datatype of this column is an object since we set strings
df.dtypes

Grades    object
dtype: object

In [4]:
#we can, however tell pandas that we want to change the type to category using the
#astype() function
df['Grades'].astype('category').head()

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
Name: Grades, dtype: category
Categories (11, object): ['A', 'A+', 'A-', 'B', ..., 'C+', 'C-', 'D', 'D+']

In [5]:
#now it is categorical, but we need to make it ordered, e.g a 'B+' should come before
#a 'B'
#we can tell pandas to order the data by first creating a new categorical data type 
#with the list of categories in order

my_categories = pd.CategoricalDtype(categories=['D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+', 'A-', 'A', 'A+'],
                                    ordered=True)
#we can pass this into the astype() function
grades=df['Grades'].astype(my_categories)
grades                                    

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
good         B-
ok           C+
ok            C
ok           C-
poor         D+
poor          D
Name: Grades, dtype: category
Categories (11, object): ['D' < 'D+' < 'C-' < 'C' ... 'B+' < 'A-' < 'A' < 'A+']

In [6]:
#now we can see that pandas is aware of the 11 categories and their order. now we
#can perfrom different tasks on it

#example on initial category before the ordering
df[df['Grades']>'C']

,Grades
ok,C+
ok,C-
poor,D+
poor,D


In [7]:
#example on final category after the ordering
grades[grades>'C']

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
good         B-
ok           C+
Name: Grades, dtype: category
Categories (11, object): ['D' < 'D+' < 'C-' < 'C' ... 'B+' < 'A-' < 'A' < 'A+']

Cut


There’s one more common scale-based operation I’d like to talk about, and that’s on converting a scale from something that is on the interval or ratio scale, like a numeric grade, into one which is categorical.
Pandas has a function called cut which takes as an argument some array-like structure like a column of a dataframe or a series. It also takes a number of bins to be used, and all bins are kept at equal spacing.

In [8]:
import numpy as np

df=pd.read_csv('census.csv')
df=df[df['SUMLEV']==50]
df=df.set_index('STNAME').groupby('STNAME')['CENSUS2010POP'].agg(np.average)
df

STNAME
Alabama                  71339.343284
Alaska                   24490.724138
Arizona                 426134.466667
Arkansas                 38878.906667
California              642309.586207
Colorado                 78581.187500
Connecticut             446762.125000
Delaware                299311.333333
District of Columbia    601723.000000
Florida                 280616.567164
Georgia                  60928.635220
Hawaii                  272060.200000
Idaho                    35626.863636
Illinois                125790.509804
Indiana                  70476.108696
Iowa                     30771.262626
Kansas                   27172.552381
Kentucky                 36161.391667
Louisiana                70833.937500
Maine                    83022.562500
Maryland                240564.666667
Massachusetts           467687.785714
Michigan                119080.000000
Minnesota                60964.655172
Mississippi              36186.548780
Missouri                 52077.626087
Monta

In [9]:
#Now we if we want to make bins, we can use cut()
pd.cut(df,10)

STNAME
Alabama                   (11706.087, 75333.413]
Alaska                    (11706.087, 75333.413]
Arizona                 (390320.176, 453317.529]
Arkansas                  (11706.087, 75333.413]
California              (579312.234, 642309.586]
Colorado                 (75333.413, 138330.766]
Connecticut             (390320.176, 453317.529]
Delaware                (264325.471, 327322.823]
District of Columbia    (579312.234, 642309.586]
Florida                 (264325.471, 327322.823]
Georgia                   (11706.087, 75333.413]
Hawaii                  (264325.471, 327322.823]
Idaho                     (11706.087, 75333.413]
Illinois                 (75333.413, 138330.766]
Indiana                   (11706.087, 75333.413]
Iowa                      (11706.087, 75333.413]
Kansas                    (11706.087, 75333.413]
Kentucky                  (11706.087, 75333.413]
Louisiana                 (11706.087, 75333.413]
Maine                    (75333.413, 138330.766]
Maryland     

Here we see that states like alabama and alaska fall into the same category, while california and the disctrict of columbia fall in a very different category.

Now, cutting is just one way to build categories from your data, and there are many other methods. For instance, cut gives you interval data, where the spacing between each category is equal sized. But sometimes you want to form categories based on frequency – you want the number of items in each bin to the be the same, instead of the spacing between bins. It really depends on what the shape of your data is, and what you’re planning to do with it.

Pivot Table


A pivot table is a way of summarizing data in a DataFrame for a particular purpose. It makes heavy use of the aggregation function. A pivot table is itself a DataFrame, where the rows represent one variable that you're interested in, the columns another, and the cell's some aggregate value. A pivot table also tends to includes marginal values as well, which are the sums for each column and row. This allows you to be able to see the relationship between two variables at just a glance.

In [10]:
import pandas as pd
import numpy as np

In [11]:
df=pd.read_csv('cwurData.csv')
df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [12]:
#Let's say we want to create a new column called Rank_Level, where institutions with 
#world ranking 1-100 are categorized as first tier and those with world ranking 
#101 - 200 are second tier, ranking 201 - 300 are third tier, after 301 is other top 
#universities.

def create_category(ranking):
    if ranking<101:
        return 'First Tier Top Unversity'
    if ranking<201:
        return 'Second Tier Top Unversity'
    if ranking<301:
        return 'Third Tier Top Unversity' 
    else:
        return 'Other Top Univerity'  

df['Rank_level']=df['world_rank'].apply(create_category)
#or use lambda
#df['Rank_level']=df['world_rank'].apply(lambda x: create_category(x))
df.head()        


,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,Rank_level
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012,First Tier Top Unversity
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012,First Tier Top Unversity
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012,First Tier Top Unversity
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012,First Tier Top Unversity
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012,First Tier Top Unversity


A pivot table allows us to pivot out one of these columns a new column headers and compare it against another column as row indices. Let's say we want to compare rank level versus country of the universities and we want to compare in terms of overall score
To do this, we tell Pandas we want the values to be Score, and index to be the country and the columns to be the rank levels. Then we specify that the aggregation function, and here we'll use the NumPy mean to get the average rating for universities in that country

In [13]:
df.pivot_table(values='score', index='country', columns='Rank_level', aggfunc=['mean']).head()

mean                      \
Rank_level First Tier Top Unversity Other Top Univerity   
country                                                   
Argentina                       NaN           44.672857   
Australia                   47.9425           44.645750   
Austria                         NaN           44.864286   
Belgium                     51.8750           45.081000   
Brazil                          NaN           44.499706   

                                                               
Rank_level Second Tier Top Unversity Third Tier Top Unversity  
country                                                        
Argentina                        NaN                      NaN  
Australia                    49.2425                47.285000  
Austria                          NaN                47.066667  
Belgium                      49.0840                46.746667  
Brazil                       49.5650                      NaN

In [14]:
df.pivot_table(values='score',index='country',columns='Rank_level',aggfunc=['mean','max']).head()

mean                      \
Rank_level First Tier Top Unversity Other Top Univerity   
country                                                   
Argentina                       NaN           44.672857   
Australia                   47.9425           44.645750   
Austria                         NaN           44.864286   
Belgium                     51.8750           45.081000   
Brazil                          NaN           44.499706   

                                                               \
Rank_level Second Tier Top Unversity Third Tier Top Unversity   
country                                                         
Argentina                        NaN                      NaN   
Australia                    49.2425                47.285000   
Austria                          NaN                47.066667   
Belgium                      49.0840                46.746667   
Brazil                       49.5650                      NaN   

                                max                      \
Rank_level First Tier Top Unversity Other Top Univerity   
country                                                   
Argentina                       NaN               45.66   
Australia                     51.61               45.97   
Austria                         NaN               46.29   
Belgium                       52.03               46.21   
Brazil                          NaN               46.08   

                                                               
Rank_level Second Tier Top Unversity Third Tier Top Unversity  
country                                                        
Argentina                        NaN                      NaN  
Australia                      50.40                    47.47  
Austria                          NaN                    47.78  
Belgium                        49.73                    47.14  
Brazil                         49.82                      NaN

As mentioned earlier, we can also summarize the values within a given top level column. For instance, if we want to see an overall average for the country for the mean and we want to see the max of the max, we can indicate that we want pandas to provide marginal values

In [15]:
df.pivot_table(values='score',index='country',columns='Rank_level',aggfunc=['mean','max'],
                    margins=True).head()

mean                      \
Rank_level First Tier Top Unversity Other Top Univerity   
country                                                   
Argentina                       NaN           44.672857   
Australia                   47.9425           44.645750   
Austria                         NaN           44.864286   
Belgium                     51.8750           45.081000   
Brazil                          NaN           44.499706   

                                                                          \
Rank_level Second Tier Top Unversity Third Tier Top Unversity        All   
country                                                                    
Argentina                        NaN                      NaN  44.672857   
Australia                    49.2425                47.285000  45.825517   
Austria                          NaN                47.066667  45.139583   
Belgium                      49.0840                46.746667  47.011000   
Brazil                       49.5650                      NaN  44.781111   

                                max                      \
Rank_level First Tier Top Unversity Other Top Univerity   
country                                                   
Argentina                       NaN               45.66   
Australia                     51.61               45.97   
Austria                         NaN               46.29   
Belgium                       52.03               46.21   
Brazil                          NaN               46.08   

                                                                      
Rank_level Second Tier Top Unversity Third Tier Top Unversity    All  
country                                                               
Argentina                        NaN                      NaN  45.66  
Australia                      50.40                    47.47  51.61  
Austria                          NaN                    47.78  47.78  
Belgium                        49.73                    47.14  52.03  
Brazil                         49.82                      NaN  49.82

In [16]:
# A pivot table is just a multi-level dataframe, and we can access series or cells in 
# the dataframe in a similar way as we do so for a regular dataframe. 
# Let's create a new dataframe from our previous example

new_df = df.pivot_table(values='score',index='country',columns='Rank_level',aggfunc=['mean','max'],
                            margins=True)
#print the index
print(new_df.index)
#print the columns 
print(new_df.columns)                           

Index(['Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Bulgaria',
       'Canada', 'Chile', 'China', 'Colombia', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Egypt', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Iran',
       'Ireland', 'Israel', 'Italy', 'Japan', 'Lebanon', 'Lithuania',
       'Malaysia', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland',
       'Portugal', 'Puerto Rico', 'Romania', 'Russia', 'Saudi Arabia',
       'Serbia', 'Singapore', 'Slovak Republic', 'Slovenia', 'South Africa',
       'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand',
       'Turkey', 'USA', 'Uganda', 'United Arab Emirates', 'United Kingdom',
       'Uruguay', 'All'],
      dtype='object', name='country')
MultiIndex([('mean',  'First Tier Top Unversity'),
            ('mean',       'Other Top Univerity'),
            ('mean', 'Second Tier Top Unversity'),
            ('mean',  'Third T

In [17]:
# We can see the columns are hierarchical. How would we query this if we want to get 
# the average scores of First Tier Top Unversity levels in each country?

new_df['mean']['First Tier Top Unversity'].head()

country
Argentina        NaN
Australia    47.9425
Austria          NaN
Belgium      51.8750
Brazil           NaN
Name: First Tier Top Unversity, dtype: float64

In [18]:
#the type would give a series because its just a single column
type(new_df['mean']['First Tier Top Unversity'])

pandas.core.series.Series

In [19]:
#to find the country wuth the maximum average score on First Tier Top University
#level, we use 'idxmax()' function

new_df['mean']['First Tier Top Unversity'].idxmax()

'United Kingdom'

Stacking and Unstacking

If you want to achieve a different shape of your pivot table, you can do so with the stack and unstack functions. Stacking is pivoting the lowermost column index to become the innermost row index. Unstacking is the inverse of stacking, pivoting the innermost row index to become the lowermost column index.

In [20]:
new_df.head()

mean                      \
Rank_level First Tier Top Unversity Other Top Univerity   
country                                                   
Argentina                       NaN           44.672857   
Australia                   47.9425           44.645750   
Austria                         NaN           44.864286   
Belgium                     51.8750           45.081000   
Brazil                          NaN           44.499706   

                                                                          \
Rank_level Second Tier Top Unversity Third Tier Top Unversity        All   
country                                                                    
Argentina                        NaN                      NaN  44.672857   
Australia                    49.2425                47.285000  45.825517   
Austria                          NaN                47.066667  45.139583   
Belgium                      49.0840                46.746667  47.011000   
Brazil                       49.5650                      NaN  44.781111   

                                max                      \
Rank_level First Tier Top Unversity Other Top Univerity   
country                                                   
Argentina                       NaN               45.66   
Australia                     51.61               45.97   
Austria                         NaN               46.29   
Belgium                       52.03               46.21   
Brazil                          NaN               46.08   

                                                                      
Rank_level Second Tier Top Unversity Third Tier Top Unversity    All  
country                                                               
Argentina                        NaN                      NaN  45.66  
Australia                      50.40                    47.47  51.61  
Austria                          NaN                    47.78  47.78  
Belgium                        49.73                    47.14  52.03  
Brazil                         49.82                      NaN  49.82

In [21]:
new_df=new_df.stack()
new_df.head()

/var/folders/1_/wks594vd2m92nsrd4rjjy3r00000gn/T/ipykernel_1198/1017776592.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  new_df=new_df.stack()


mean    max
country   Rank_level                                 
Argentina Other Top Univerity        44.672857  45.66
          All                        44.672857  45.66
Australia First Tier Top Unversity   47.942500  51.61
          Other Top Univerity        44.645750  45.97
          Second Tier Top Unversity  49.242500  50.40

In [22]:
new_df.unstack().head()


mean                      \
Rank_level First Tier Top Unversity Other Top Univerity   
country                                                   
All                       58.350675           44.738871   
Argentina                       NaN           44.672857   
Australia                 47.942500           44.645750   
Austria                         NaN           44.864286   
Belgium                   51.875000           45.081000   

                                                                          \
Rank_level Second Tier Top Unversity Third Tier Top Unversity        All   
country                                                                    
All                         49.06545                46.843450  47.798395   
Argentina                        NaN                      NaN  44.672857   
Australia                   49.24250                47.285000  45.825517   
Austria                          NaN                47.066667  45.139583   
Belgium                     49.08400                46.746667  47.011000   

                                max                      \
Rank_level First Tier Top Unversity Other Top Univerity   
country                                                   
All                          100.00               46.34   
Argentina                       NaN               45.66   
Australia                     51.61               45.97   
Austria                         NaN               46.29   
Belgium                       52.03               46.21   

                                                                       
Rank_level Second Tier Top Unversity Third Tier Top Unversity     All  
country                                                                
All                            51.29                    47.93  100.00  
Argentina                        NaN                      NaN   45.66  
Australia                      50.40                    47.47   51.61  
Austria                          NaN                    47.78   47.78  
Belgium                        49.73                    47.14   52.03

In [23]:
new_df.unstack().head().unstack()

      Rank_level                 country  
mean  First Tier Top Unversity   All           58.350675
                                 Argentina           NaN
                                 Australia     47.942500
                                 Austria             NaN
                                 Belgium       51.875000
      Other Top Univerity        All           44.738871
                                 Argentina     44.672857
                                 Australia     44.645750
                                 Austria       44.864286
                                 Belgium       45.081000
      Second Tier Top Unversity  All           49.065450
                                 Argentina           NaN
                                 Australia     49.242500
                                 Austria             NaN
                                 Belgium       49.084000
      Third Tier Top Unversity   All           46.843450
                                 Argentina   